In [1]:
import numpy as np
import GPy
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from mpl_toolkits import mplot3d
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
def dataMaker(version):
    df = pd.read_csv ('realData/uq_vsd_case' + version + '_trenddata.csv', error_bad_lines=False, index_col = False )

    df = df.loc[:,['RelativeTimeMilliseconds', 'Pulse', 'RR']]
    
    nan_val = float("NaN")
    df = df.replace("",nan_val)
    df = df.dropna(axis=0, how='any')
    df = df.reset_index(drop=True)

    df = df.loc[:580]

    df['RelativeTimeMilliseconds'] = np.arange(0, 581, 1)

    print(df['RelativeTimeMilliseconds'])
    
    df[['RelativeTimeMilliseconds', 'Pulse', 'RR']] = StandardScaler().fit_transform(df[['RelativeTimeMilliseconds', 'Pulse', 'RR']])

    #StandardScaler().fit_transform(arr['Pulse'].values.reshape(-1, 1))

  
    

    print(df)

    return df


In [3]:
%%capture --no-display

data = []

for i in range(1,33):
    temp = str(i)
    if i < 10:
        data.append(dataMaker('0' + temp))
    elif i in [17,18,19]:
        continue
    else:
        data.append(dataMaker(temp))

#data now contains the first 580 values for each subject with, time, pulse and rr


In [4]:
#here: loop through data list, fit a model for each attibute against time, and save those model (2 in total for each subject) in a separate list. 

#the final list should be a list of list, where the outer list is 32 long and each index contains two models for each subject.

print(len(data))

counter = 0
for item in data:
    counter +=1
    if item.isnull().sum().values.sum() != 0:
        print("version: " , counter)
        print("Data: ")
        print(item.isnull().sum())
        print("-------")

29


In [5]:
def modelMaker(arr):

    X_axis = arr['RelativeTimeMilliseconds'].values.reshape(-1, 1)
    Y_axis = arr['Pulse'].values.reshape(-1, 1)
    Z_axis = arr['RR'].values.reshape(-1, 1)

    #------Y-model------------
    print("Ymodel: ")
    kernelY = GPy.kern.RBF(input_dim=1)
    Ymodel = GPy.models.GPRegression(X_axis, Y_axis, kernelY)

    Ymodel.optimize(messages=True, ipython_notebook=True, clear_after_finish=True)
    Ymodel.optimize_restarts(num_restarts = 2)

    #------Z-model-----------
    print("Zmodel: ")

    kernelZ = GPy.kern.RBF(input_dim=1)
    Zmodel = GPy.models.GPRegression(X_axis, Z_axis, kernelZ)

    Zmodel.optimize(messages=True, ipython_notebook=True, clear_after_finish=True)
    Zmodel.optimize_restarts(num_restarts = 2)

    model = {
        "Ymodel" : Ymodel,
        "Zmodel" : Zmodel
    }

    return model

In [6]:
modelList = []
count = 0
for i in data:
    count = count + 1
    print(count) 
    modelList.append(modelMaker(i))

1
Ymodel: 
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0002   6.138053e+02   2.720728e+03 
    01s10  0032   9.515219e+01   4.525499e-11 
Runtime:     01s10
Optimization status: Converged

Optimization restart 1/2, f = 95.15219381489749
Optimization restart 2/2, f = 95.15219381490073
Zmodel: 
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s12  0002   6.768609e+02   1.358327e+04 
    00s19  0004   1.217144e+02   1.477772e+05 
    00s71  0020  -3.748939e+01   6.352190e-08 
Runtime:     00s71
Optimization status: Converged

Optimization restart 1/2, f = -37.48939453748932
Optimization restart 2/2, f = -37.48939453752217
2
Ymodel: 
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s00  0000   6.225131e+02           nan 
    00s19  0004   1.102765e+03   1.595737e+06 
    01s20  0042  -9.526560e+01   5.741970e+04 
    01s61  0052  -1.41

 /home/fabs/code/astraVenv/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /home/fabs/code/astraVenv/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /home/fabs/code/astraVenv/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/2, f = 121.17850452001971
Zmodel: 
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0002   4.316765e+02   2.707993e+04 
    00s68  0023  -5.538549e+02   2.649225e-08 
Runtime:     00s68
Optimization status: Converged

Optimization restart 1/2, f = -553.8548988411671
Optimization restart 2/2, f = -553.8548988411644
6
Ymodel: 
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s17  0003   1.114974e+03   4.101067e+06 
    00s73  0025  -3.170897e+02   1.538640e-07 
Runtime:     00s73
Optimization status: Converged

Optimization restart 1/2, f = -317.08968915441017
Optimization restart 2/2, f = -317.0896891547677
Zmodel: 
Running L-BFGS-B (Scipy implementation) Code:
  runtime   i      f              |g|        
    00s16  0003   2.007387e+03   6.562876e+06 
    00s70  0030  -5.437218e+02   1.697999e-08 
Runtime:     00s70
Optimization status: Converged

Optimization r

In [7]:
print(len(modelList))

29


In [48]:
import scipy.stats as st
import math

sMatrix = np.empty((len(data),len(data)))
print(sMatrix.shape)

#method found in:
#https://dfrieds.com/math/z-scores.html

#get mean and variance from datapoint 3 in a dataset
mu, sigma = modelList[0]["Ymodel"].predict_noiseless(data[0]['RelativeTimeMilliseconds'].loc[:3].values.reshape(-1,1))
print(mu)
print(sigma)


#calculate z score to be used later in the probability calculations
z = (data[0]['Pulse'].loc[:3].values.reshape(-1,1) - mu) / sigma
print(z)

#probability calculation using scipy
p = st.norm.cdf(z)
print(p)
tot = 1
for num in p:
    tot *= num

print(tot)

prob = -math.log(tot)

print(prob)


#same steps as described above
mu, sigma = modelList[0]["Ymodel"].predict_noiseless(data[0]['RelativeTimeMilliseconds'].loc[1:1].values.reshape(-1,1))
print("mu")
print(mu)
print("sigma")
print(math.sqrt(sigma))

z = (data[0]['Pulse'].loc[1:1].values.reshape(-1,1) - mu) / sigma
print("z score")
print(z)


p = st.norm.cdf(z)
print("p")
print(p)
tot = 1
for num in p:
    tot *= num

print("Tot")
print(tot)

prob = -math.log(tot)

print("prob")
print(prob)


mu, sigma = modelList[0]["Ymodel"].predict_noiseless(data[2]['RelativeTimeMilliseconds'].loc[1:1].values.reshape(-1,1))
print("mu")
print(mu)
print("sigma")
print(math.sqrt(sigma))

z = (data[2]['Pulse'].loc[1:1].values.reshape(-1,1) - mu) / sigma
print("z score")
print(z)


p = st.norm.cdf(z)
print("p")
print(p)
tot = 1
for num in p:
    tot *= num

#attemp to compare data from case 2 to model generatod for case1. 
#the comparison is either 1 or 0. nothing in between. 

print("Tot")
print(tot)

prob2 = -math.log(tot)

print("prob2")
print(prob2)


print("difference")
print(prob2/prob)

#----------------------------------------------
#new attempt starting here trying out:
#https://www.mhnederlof.nl/bayesnormalupdate.html






        



(29, 29)
[[-1.26565716]
 [-1.14765409]
 [-1.22776118]
 [-1.80367625]]
[[0.01501139]
 [0.00794348]
 [0.00794339]
 [0.00739446]]
[[  2.48754992]
 [-10.15442587]
 [-30.90101223]
 [ 44.68970955]]
[[9.93568680e-001]
 [1.58334253e-024]
 [5.78818735e-210]
 [1.00000000e+000]]
[9.10574222e-234]
536.596006532757
mu
[[-1.14765409]]
sigma
0.08912620021889772
z score
[[-10.15442587]]
p
[[1.58334253e-24]]
Tot
[1.58334253e-24]
prob
54.80250409134087
mu
[[-1.14765409]]
sigma
0.08912620021889772
z score
[[49.66692106]]
p
[[1.]]
Tot
[1.]
prob2
-0.0
difference
-0.0


In [9]:
print(modelList[0]["Ymodel"].to_dict()["Y"][0:4])
print(data[0]['Pulse'].loc[0:3].values)
print("Time")
#print(data[0]['RelativeTimeMilliseconds'].loc[0:3].values)
#print(data[1]['RelativeTimeMilliseconds'].loc[0:3].values)

print(modelList[0]["Ymodel"].predict_noiseless(data[0]['RelativeTimeMilliseconds'].loc[0:3].values.reshape(-1,1)))
print(modelList[0]["Ymodel"].predict_noiseless(data[4]['RelativeTimeMilliseconds'].loc[0:3].values.reshape(-1,1)))


[[-1.2283155688981622], [-1.2283155688981622], [-1.4732199427930943], [-1.4732199427930943]]
[-1.22831557 -1.22831557 -1.47321994 -1.47321994]
Time
(array([[-1.26565716],
       [-1.14765409],
       [-1.22776118],
       [-1.80367625]]), array([[0.01501139],
       [0.00794348],
       [0.00794339],
       [0.00739446]]))
(array([[-1.26565716],
       [-1.14765409],
       [-1.22776118],
       [-1.80367625]]), array([[0.01501139],
       [0.00794348],
       [0.00794339],
       [0.00739446]]))


In [10]:
print(data[0]['Pulse'].loc[0:0].values.reshape(-1,1))

[[-1.22831557]]


In [11]:
#new